# Konfigurasi dan Instalasi
Cell ini berisi instalasi paket yang diperlukan dan pengaturan konfigurasi untuk model TimesFM. Semua parameter hard-coded didefinisikan di sini untuk memudahkan pengelolaan.

In [1]:
# Instalasi paket yang diperlukan
!pip install timesfm[torch] pandas numpy seaborn -q

# Import pustaka
import pandas as pd
import numpy as np
import seaborn as sns
import timesfm
from timesfm import TimesFm

# Konfigurasi model TimesFM
MODEL_CONFIG = {
    'backend': 'gpu',
    'per_core_batch_size': 32,
    'horizon_len': 24,
    'num_layers': 50,
    'use_positional_embedding': False,
    'context_len': 2048,
    'huggingface_repo_id': 'google/timesfm-2.0-500m-pytorch'
}

# Konfigurasi data
DATA_CONFIG = {
    'url': 'https://github.com/evanhfw/IEEE-code/raw/refs/heads/main/data/Finlandia.csv',
    'columns': ['Consumption', 'Date'],
    'new_columns': ['y', 'ds'],
    'freq': 'h',
    'num_folds': 20,
    'train_end_idx': 17015,
    'unique_id': 1
}

# Konfigurasi output
OUTPUT_CONFIG = {
    'test_output_file': 'Data Test Ori - Fold.csv',
    'result_output_file': 'Result TIMEFM - Finlandia.csv'
}

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━

# Memuat dan Mempersiapkan Data
Bagian ini bertujuan untuk memuat data dari URL yang diberikan, mengubah nama kolom, dan menambahkan kolom `unique_id` untuk keperluan model TimesFM.

In [2]:
# Memuat data dari URL
finland = pd.read_csv(DATA_CONFIG['url'])
finland = finland[DATA_CONFIG['columns']]
finland.columns = DATA_CONFIG['new_columns']

# Mengubah kolom 'ds' menjadi format datetime
finland['ds'] = pd.to_datetime(finland['ds'])

# Menambahkan kolom unique_id
finland['unique_id'] = DATA_CONFIG['unique_id']

# Menampilkan informasi dasar dataset
print(f"Jumlah baris data: {len(finland)}")
print(f"Kolom dataset: {finland.columns.tolist()}")
print(f"Rentang tanggal: {finland['ds'].min()} hingga {finland['ds'].max()}")

Jumlah baris data: 17520
Kolom dataset: ['y', 'ds', 'unique_id']
Rentang tanggal: 2022-01-01 01:00:00 hingga 2024-01-01 00:00:00


# Inisialisasi Model TimesFM
Cell ini menginisialisasi model TimesFM dengan parameter yang telah didefinisikan di konfigurasi.

In [3]:
# Inisialisasi model TimesFM
tfm = TimesFm(
    hparams=timesfm.TimesFmHparams(
        backend=MODEL_CONFIG['backend'],
        per_core_batch_size=MODEL_CONFIG['per_core_batch_size'],
        horizon_len=MODEL_CONFIG['horizon_len'],
        num_layers=MODEL_CONFIG['num_layers'],
        use_positional_embedding=MODEL_CONFIG['use_positional_embedding'],
        context_len=MODEL_CONFIG['context_len'],
    ),
    checkpoint=timesfm.TimesFmCheckpoint(
        huggingface_repo_id=MODEL_CONFIG['huggingface_repo_id']
    )
)

print("Model TimesFM berhasil diinisialisasi.")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

torch_model.ckpt:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Model TimesFM berhasil diinisialisasi.


# Pembuatan Data Uji
Bagian ini membuat data uji dengan membagi data menjadi beberapa fold berdasarkan konfigurasi. Data uji disimpan dalam format DataFrame dan diekspor ke file CSV.

In [6]:
# Inisialisasi list untuk menyimpan data uji
tests = []
tests_df = pd.DataFrame()

# Membuat data uji untuk setiap fold
for i in range(DATA_CONFIG['num_folds']):
    start_idx = DATA_CONFIG['train_end_idx'] + i * MODEL_CONFIG['horizon_len']
    end_idx = start_idx + MODEL_CONFIG['horizon_len']
    test = finland.iloc[start_idx:end_idx]
    tests.append(test)
    tests_df[f'fold_{i}'] = test['y'].values
    print(f"Data uji fold {i} dibuat dengan {len(test)} baris.")

# Menyimpan data uji ke file CSV
tests_df.to_csv(OUTPUT_CONFIG['test_output_file'], index=False)
print(f"Data uji disimpan ke {OUTPUT_CONFIG['test_output_file']}")

Data uji fold 0 dibuat dengan 24 baris.
Data uji fold 1 dibuat dengan 24 baris.
Data uji fold 2 dibuat dengan 24 baris.
Data uji fold 3 dibuat dengan 24 baris.
Data uji fold 4 dibuat dengan 24 baris.
Data uji fold 5 dibuat dengan 24 baris.
Data uji fold 6 dibuat dengan 24 baris.
Data uji fold 7 dibuat dengan 24 baris.
Data uji fold 8 dibuat dengan 24 baris.
Data uji fold 9 dibuat dengan 24 baris.
Data uji fold 10 dibuat dengan 24 baris.
Data uji fold 11 dibuat dengan 24 baris.
Data uji fold 12 dibuat dengan 24 baris.
Data uji fold 13 dibuat dengan 24 baris.
Data uji fold 14 dibuat dengan 24 baris.
Data uji fold 15 dibuat dengan 24 baris.
Data uji fold 16 dibuat dengan 24 baris.
Data uji fold 17 dibuat dengan 24 baris.
Data uji fold 18 dibuat dengan 24 baris.
Data uji fold 19 dibuat dengan 24 baris.
Data uji disimpan ke Data Test Ori - Fold.csv


# Proses Forecasting
Bagian ini melakukan forecasting menggunakan model TimesFM untuk setiap fold dan menyimpan hasilnya ke dalam DataFrame.

In [7]:
# Inisialisasi list untuk menyimpan hasil forecasting
results = []
results_df = pd.DataFrame()

# Melakukan forecasting untuk setiap fold
for i in range(DATA_CONFIG['num_folds']):
    start_idx = i * MODEL_CONFIG['horizon_len']
    end_idx = DATA_CONFIG['train_end_idx'] + i * MODEL_CONFIG['horizon_len']
    input_data = finland.iloc[start_idx:end_idx]
    
    print(f"Memproses forecasting untuk fold {i} dengan {len(input_data)} baris input.")
    
    # Melakukan forecasting
    forecast_df = tfm.forecast_on_df(
        inputs=input_data,
        freq=DATA_CONFIG['freq'],
        value_name='y',
        num_jobs=-1
    )
    results.append(forecast_df)
    results_df[f'fold_{i}'] = forecast_df['timesfm']
    print(f"Forecasting fold {i} selesai.")

# Menyimpan hasil forecasting ke file CSV
results_df.to_csv(OUTPUT_CONFIG['result_output_file'], index=False)
print(f"Hasil forecasting disimpan ke {OUTPUT_CONFIG['result_output_file']}")

Memproses forecasting untuk fold 0 dengan 17015 baris input.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Forecasting fold 0 selesai.
Memproses forecasting untuk fold 1 dengan 17015 baris input.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Forecasting fold 1 selesai.
Memproses forecasting untuk fold 2 dengan 17015 baris input.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Forecasting fold 2 selesai.
Memproses forecasting untuk fold 3 dengan 17015 baris input.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Forecasting fold 3 selesai.
Memproses forecasting untuk fold 4 dengan 17015 baris input.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Forecasting fold 4 selesai.
Memproses forecasting untuk fold 5 dengan 170